# AI_CHOICE

## Overview
AI_CHOICE uses AI to classify or select the most appropriate option from a list of choices based on provided text. It is ideal for categorization, sentiment analysis, prioritization, and other decision-making tasks where multiple options must be evaluated against a text description.

## Usage
To use the function in Excel:
```excel
=AI_CHOICE(text, choices, [temperature], [model], [api_key], [api_url])
```

## Arguments
| Argument     | Type           | Required | Description                                                                                                 | Example                                      |
|--------------|----------------|----------|-------------------------------------------------------------------------------------------------------------|----------------------------------------------|
| text         | string/2D list | Yes      | The text to classify or analyze.                                                                            | "Uber ride from airport to hotel, $45.50"   |
| choices      | string/2D list | Yes      | The options to choose from (comma-separated string or range with one option per cell).                      | {"Travel", "Food", "Office", "Software"} |
| temperature  | float          | No       | Controls the randomness/creativity of the response (0.0 to 2.0). Higher values mean more creative.          | 0.2                                          |
| model        | string         | No       | The specific AI model ID to use for the request (e.g., 'mistral-small', 'mistral-large').                   | "mistral-small-latest"                      |
| api_key      | string         | No       | API key for authentication. [Get a free API key from Mistral AI](https://console.mistral.ai/).              | "sk-..."                                    |
| api_url      | string         | No       | OpenAI-compatible API endpoint URL which supports CORS.                                                     | "https://api.mistral.ai/v1/chat/completions"|

## Return Value
| Return Value | Type   | Description                                  | Example         |
|--------------|--------|----------------------------------------------|-----------------|
| Result       | string | The selected choice from the provided options | "Travel"       |
| Error        | string | Error message if input is invalid             | "Error: ..."   |

## Limitations
- The quality of the result depends on the clarity of the text and the options provided.
- Large lists or long text may exceed model context limits and result in truncated or incomplete responses.
- The function requires an internet connection to access the AI model.
- Model availability and output may vary depending on the provider or API changes.
- Sensitive or confidential data should not be sent to external AI services.
- `temperature` must be a float between 0 and 2 (inclusive). If not, an error message is returned.
- If you hit the API rate limit for your provider, a message is returned instead of raising an exception.

## Examples

### Expense Categorization
| Input Text                                 | Choices                                 |
|--------------------------------------------|------------------------------------------|
| Uber ride from airport to hotel, $45.50    | Travel, Food, Office, Software           |

```excel
=AI_CHOICE(A1, B1:B4)
```
**Sample Output:**
"Travel"

### Email Sentiment Analysis
| Input Text                                                                                                   | Choices                        |
|-------------------------------------------------------------------------------------------------------------|---------------------------------|
| I've been waiting for a response about my refund for over two weeks now. This is completely unacceptable...  | Positive, Neutral, Negative    |

```excel
=AI_CHOICE(A1, B1:B3)
```
**Sample Output:**
"Negative"

In [6]:
import requests
import json

def ai_choice(text, choices, temperature=0.2, model='mistral-small-latest', api_key=None, api_url="https://api.mistral.ai/v1/chat/completions"):
    """
    Uses AI to select the most appropriate choice from a list of options based on the given context.

    Args:
        text (str or 2D list): The context, question, or scenario used for decision-making
        choices (str or 2D list): A string with comma-separated options or a 2D list of options
        temperature (float, optional): Controls randomness in the selection (0-2). Default is 0.2
        model (str, optional): ID of the AI model to use
        api_key (str, optional): API key for authentication (e.g. for Mistral AI)
        api_url (str, optional): OpenAI compatible URL. (e.g., https://api.mistral.ai/v1/chat/completions for Mistral AI).

    Returns:
        str: The selected choice from the options provided, or an error message string if input is invalid
    """
    # Demo login fallback
    if api_key is None or api_url is None:
        if 'idToken' in globals():
            api_key = globals()['idToken']
            api_url = "https://llm.boardflare.com"
        else:
            return ("Login on the Functions tab for limited demo usage, or sign up for a free Mistral AI account at https://console.mistral.ai/ and add your own api_key.")

    # Input validation for temperature
    if not isinstance(temperature, (float, int)) or not (0 <= float(temperature) <= 2):
        return "Error: temperature must be a float between 0 and 2 (inclusive)"

    # Normalize text to string if it's a 2D list
    if isinstance(text, list):
        text_str = "\n".join([item[0] if isinstance(item, list) and len(item) > 0 else str(item) for item in text if len(item) > 0])
    else:
        text_str = text

    # Normalize choices to a list of strings
    if isinstance(choices, list):
        choices_list = [item[0] if isinstance(item, list) and len(item) > 0 else str(item) for item in choices]
    else:
        choices_list = [choice.strip() for choice in str(choices).split(',') if choice.strip()]

    if not text_str or text_str.strip() == "":
        return "Error: Empty input text."
    if not choices_list or all([c.strip() == "" for c in choices_list]):
        return "Error: No valid choices provided."

    # Construct the AI prompt
    prompt = f"""Based on the following context, select the single most appropriate option from the choices provided.\n\nContext:\n{text_str}\n\nChoices:\n{json.dumps(choices_list, indent=2)}\n\nProvide ONLY your selected choice without explanation or additional text. Return the exact text of the selected choice."""

    # Prepare the API request payload
    payload = {
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "model": model,
        "max_tokens": 200
    }

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    # Make the API request
    try:
        response = requests.post(api_url, headers=headers, json=payload)
        if response.status_code == 429:
            return "You have hit the rate limit for the API. Please try again later."
        response_data = response.json()
        content = response_data["choices"][0]["message"]["content"].strip()
        # Validate that the response is one of the choices
        for choice in choices_list:
            if choice == content:
                return choice
        # If no exact match, return the AI's response (which may be a paraphrase)
        return content
    except Exception as e:
        return f"Error: Failed to get AI recommendation. {str(e)}"

In [7]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent / "test"))
from test_utils import get_graph_token

def inject_id_token():
    token = get_graph_token()
    globals()["idToken"] = token

inject_id_token()

def test_expense_categorization():
    text = "Uber ride from airport to hotel, $45.50"
    choices = ["Travel", "Food", "Office", "Software"]
    result = ai_choice(text, choices)
    assert isinstance(result, str)
    assert result

def test_email_sentiment():
    text = "I've been waiting for a response about my refund for over two weeks now. This is completely unacceptable and I'm considering filing a complaint."
    choices = ["Positive", "Neutral", "Negative"]
    result = ai_choice(text, choices)
    assert isinstance(result, str)
    assert result

def test_empty_choices():
    text = "Test with no choices"
    choices = []
    result = ai_choice(text, choices)
    assert isinstance(result, str)
    assert "Error" in result

def test_empty_text():
    text = ""
    choices = ["Yes", "No"]
    result = ai_choice(text, choices)
    assert isinstance(result, str)
    assert "Error" in result

ipytest.run()

....                                                                                         [100%]
4 passed in 0.23s
...                                                                                         [100%]
4 passed in 0.23s


<ExitCode.OK: 0>

In [ ]:
# Gradio Demo
import gradio as gr

examples = [
    [
        "Uber ride from airport to hotel, $45.50",
        [["Travel"], ["Food"], ["Office"], ["Software"]],
        0.2,
        "mistral-small-latest"
    ],
    [
        "I've been waiting for a response about my refund for over two weeks now. This is completely unacceptable and I'm considering filing a complaint.",
        [["Positive"], ["Neutral"], ["Negative"]],
        0.2,
        "mistral-small-latest"
    ]
]

demo = gr.Interface(
    fn=ai_choice,
    inputs=[
        gr.Textbox(label="Text to Analyze", lines=2),
        gr.Dataframe(label="Choices (one per row)", type="array", headers=None, datatype="str", row_count=(1, "dynamic"), col_count=(1, "fixed")),
        gr.Slider(0.0, 2.0, value=0.2, step=0.01, label="Temperature"),
        gr.Textbox(value="mistral-small-latest", label="Model")
    ],
    outputs=gr.Textbox(label="Selected Choice"),
    examples=examples,
    description="Classify or select the most appropriate option from a list of choices based on provided text. Enter the text to analyze and a list of choices (one per row). Adjust the temperature for creativity and specify the model if desired.",
    flagging_mode="never",
)
demo.launch()

c:\Users\brent\Code\python-apps\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
